# 12-1 들어가면서

학습목표

1. 번역 성능을 측정하는 BLUE score와 Beam Search Decoder를 익히고

2. chatbot을 만들어보기 

파일 경로

``` teriminal
$ mkdir -p ~/aiffel/transformer_chatbot
```



Ubuntu KoNLPy 설치

``` terminal
$ sudo apt-get install g++ openjdk-8-jdk
$ sudo apt-get install curl
$ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
$ pip install konlpy
```

## library와 데이터

In [79]:
import numpy as np
import pandas as pd
import tensorflow as tf
from konlpy.tag import Mecab
import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.2.0


## Data 다운로드

[데이터 링크](https://github.com/songys/Chatbot_data)




In [80]:
data = pd.read_csv("ChatbotData .csv")

In [81]:
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


Q,A label로 이루어져있는데 0은 일상다반서, 이별(부정)은 1, 사랑(긍정)은 2로 labeling 되어있음. 
우리에게는 필요없는 정보라 제외함

In [82]:
questions = data.Q
ansewers = data.A

In [83]:
questions

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [84]:
ansewers

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

## 데이터 정제

In [85]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z0-9ㄱ-ㅎ|가-힣?.!,]+", " ", sentence)  # a-zA-Z0-9ㄱ-ㅎ|가-힣?.! 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    return sentence

print(preprocess_sentence("This is sample # $ %       sentence."))   # 이 문장이 어떻게 필터링되는지 되는것을 통해 확인

this is sample sentence.


## 데이터 토큰화

In [86]:
def build_corpus(s_s,t_s, length = 25):
    que_corpus = [] 
    ans_corpus = []
    try:
        if len(s_s) != len(t_s):
            raise Exception('소스 문장 데이터와 타켓 문장 데이터의 길이가 다릅니다.')
    except Exception as e:                             # 예외가 발생했을 때 실행됨
        print(f'소스 문장 데이터의 길이 {len(s_s)} 타켓 문장 데이터의 길이 {len(t_s)} 입니다.', e)
        
    for i in range(len(s_s)):
        mecab = Mecab()
        q, a = s_s[i], t_s[i]
        q = preprocess_sentence(q)
        q = mecab.morphs(q)
        
        a = preprocess_sentence(a)
        a = mecab.morphs(a)
        
        if (len(q) < length) & (len(a) < length) : # 길이 제외
            if (q not in que_corpus) & (a not in ans_corpus) : 
                que_corpus.append(q)
                ans_corpus.append(a)
    return que_corpus, ans_corpus

함수를 짤때 문장의 길이가 다르면 에러 처리를 해주게 만들어보았다.

In [128]:
que_corpus, ans_corpus = build_corpus(ansewers[:1],questions)

소스 문장 데이터의 길이 1 타켓 문장 데이터의 길이 11823 입니다. 소스 문장 데이터와 타켓 문장 데이터의 길이가 다릅니다.


In [129]:
que_corpus, ans_corpus = build_corpus(questions,ansewers)

In [130]:
que_corpus[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네'],
 ['sd', '카드', '망가졌', '어']]

In [131]:
ans_corpus[:5]


[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']]

## Augmentation

``` terminal
$ pip install gensim
```


CV에만 있는줄알았는데 NLP에도 존재한다. 데이터가 1만개 정도 됨으로 augmentation 기법        
그중에서도 **Embdedding을 활용한 Lexical Subsitution**을 구현해보도록 한다.       

아래 링크에서 한국어로 사전 훈련된 Embedding 모델을 다운로드한다.
Korean(w)가 Word2Vec으로 학습한 모델이며      
용량이 적당함으로 다운로드해서 ko.bin 파일을 얻는다.    

[Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) 

우리의 목표는 데이터를 ans_corpus의 aug를 통해 2배로 늘리고 que_corpus의 aug를 통해 2배로 늘려서 

총 양을 3배 정도로 늘리는 것

In [132]:
import gensim
import os
from tqdm.notebook import tqdm

In [133]:
kobin_path = os.getenv('HOME')+'/aiffel/transformer_chatbot/ko.bin'
word2vec = gensim.models.Word2Vec.load(kobin_path)

In [134]:
def lexical_sub(sentence, word2vec):
    import random

    res = []
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res


In [135]:
print(len(que_corpus), len(ans_corpus))

7648 7648


In [136]:
que_corpus_aug = que_corpus
ans_corpus_aug = ans_corpus
for idx in range(len(que_corpus)):
    q = lexical_sub(que_corpus[idx],word2vec)
    a = lexical_sub(ans_corpus[idx],word2vec)
    if a != None:
        que_corpus_aug.append(que_corpus[idx])
        ans_corpus_aug.append(a)
    if q != None:
        que_corpus_aug.append(q)
        ans_corpus_aug.append(ans_corpus[idx])

/home/ssac4/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


아래와 같이 augmentation된게 추가되었다.

In [141]:
que_corpus[0]

['12', '시', '땡', '!']

In [140]:
que_corpus_aug[7649]

['12', '시', '끗', '!']

In [151]:
ans_corpus[2]

['여행', '은', '언제나', '좋', '죠', '.']

In [144]:
ans_corpus_aug[7650]

['여행', '은', '항상', '좋', '죠', '.']

In [152]:
print(len(ans_corpus_aug),len(que_corpus))

20953 20953


## 데이터 벡터화

## BLUE Score 

Bilingual Evaluation Understudy score 

[참고 자료](https://donghwa-kim.github.io/BLEU.html)

In [2]:
# !pip install nltk # nltk가 설치되어 있지 않은 경우 주석 해제
from nltk.translate.bleu_score import sentence_bleu

reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 8.190757052088229e-155


/home/ssac4/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ssac4/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [3]:
print("1-gram:", sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))
print("2-gram:", sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))
print("3-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))
print("4-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))

1-gram: 0.5
2-gram: 0.18181818181818182
3-gram: 2.2250738585072626e-308
4-gram: 2.2250738585072626e-308


$$
(\prod_{i=1}^4 precision_i)^{\frac{1}{4}} = (\text{1-gram} \times\text{2-gram} \times\text{3-gram} \times\text{4-gram})^{\frac{1}{4}}
$$

3gram, 4gram에서 0에 가까운 점수를 받아서 BLUE SCORE가 지나치게 낮음.

곱연산후 루트를 씌우기때문에 하나의 값이 0이면 지나치게 총 BLUE가 작아지기 때문

가끔 0으로 계산되는 값을 1.0으로 보정해서 게산하기도 했는데 1.0은 완벽한 번역을의미하기때문에 총점이 이상하게 높아질수가 있음.

즉 0,1 의 값들 때문에 값이 왜곡되는 경우가 많았음. 이를 방지하기위해

이런 걸 방지하기 위해 SmoothingFunction으로 epsilon을 더해줘서 0에 가까운 값도 보정해줘서 해결함.


In [4]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


이제 조금 더 해석하기 편리하게 BLUE 계산되었음 이때 사용하는 SmothingFunction()은 디폴트가 mothod0이며 7까지 존재함

In [12]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="spa-eng",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang

    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer
print("슝=3")

슝=3


## Beam Search Decoder


In [5]:
import math

def beam_search_decoder(prob, beam_size):
    sequences = [[[], 1.0]]  # 생성된 문장과 점수를 저장

    for tok in prob:
        all_candidates = []

        for seq, score in sequences:
            for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
                candidate = [seq + [idx], score * -math.log(-(p-1))]
                all_candidates.append(candidate)

        ordered = sorted(all_candidates,
                         key=lambda tup:tup[1],
                         reverse=True) # 총점 순 정렬
        sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 

    return sequences
print("슝=3")

슝=3


In [6]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17],
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01],
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68],
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01],
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
    sentence = ""

    for word in seq:
        sentence += vocab[word] + " "

    print(sentence, "// Score: %.4f" % score)

커피 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 42.5243
커피 를 마셔 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 28.0135
마셔 를 가져 도 될 까요? <pad> <pad> <pad> <pad>  // Score: 17.8983


사실 위 예시는 Beam Search 설명에는 적당한데 실제 모델이 문장 생성하는 과정과는 거리가 멈.     
prob_seq처럼 확률을 정의하기가 힘듬. 한번에 정의되지않고 이전 스텝의 단어에 따라서 결정되기 때문임.     
지금은 그런걸 고려하지않고 그냥 3번째 단어는 마신다 0.3 먹는다 0.5,... 의 확률을 가진다고 적어놨음.    
  
그래서 실제로 Beam Search를 생성긱법으로 구현할때는 분기를 잘 나눠줘어야함.

In [8]:
def calc_prob(src_ids, tgt_ids, model):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(src_ids, tgt_ids)

    predictions, enc_attns, dec_attns, dec_enc_attns =\
    model(src_ids, 
            tgt_ids,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask)

    return tf.math.softmax(predictions, axis=-1)

In [9]:
def beam_search_decoder(sentence, 
                        src_len,
                        tgt_len,
                        model,
                        src_tokenizer,
                        tgt_tokenizer,
                        beam_size):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    src_in = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                            maxlen=src_len,
                                                            padding='post')

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.long)
    pred = np.zeros((beam_size, tgt_len), dtype=np.long)

    eos_flag = np.zeros((beam_size, ), dtype=np.long)
    scores = np.ones((beam_size, ))

    pred[:, 0] = tgt_tokenizer.bos_id()

    dec_in = tf.expand_dims(pred[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size, ))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            score_cache[cache_pos:cache_pos+beam_size] = scores[branch_idx]
            pred_cache[cache_pos:cache_pos+beam_size, :seq_pos] = \
            pred[branch_idx, :seq_pos]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            if seq_pos != 1:   # 모든 Branch를 <BOS>로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos+beam_idx] *= prob[max_idx]
                pred_cache[cache_pos+beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1: continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, :seq_pos+1]

            pred[beam_idx, :seq_pos+1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == tgt_tokenizer.eos_id():
                eos_flag[beam_idx] = -1

    return pred

In [10]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                            candidate,
                            weights=weights,
                            smoothing_function=SmoothingFunction().method1)

def beam_bleu(reference, ids, tokenizer):
    reference = reference.split()

    total_score = 0.0
    for _id in ids:
        candidate = tokenizer.decode_ids(_id.tolist()).split()
        score = calculate_bleu(reference, candidate)

        print("Reference:", reference)
        print("Candidate:", candidate)
        print("BLEU:", calculate_bleu(reference, candidate))

        total_score += score

    return total_score / len(ids)

In [11]:
idx = 324

ids = \
beam_search_decoder(tokenizer.decode_ids(enc_val[idx].tolist()),
                    enc_train.shape[-1],
                    dec_train.shape[-1],
                    transformer,
                    tokenizer,
                    tokenizer,
                    beam_size=5)

bleu = beam_bleu(tokenizer.decode_ids(dec_val[idx].tolist()), ids, tokenizer)

NameError: name 'tokenizer' is not defined

## 소고

 BLEU를 계산할때, 의미가 비슷한 단어(임베딩 벡터에서 유사한 단어, 혹은 동음이의어)는 높게, 아예 의미가 다른 단어면 낮은점수를 주는 방법은 없을까?